In [2]:
%load_ext autoreload
%autoreload 2

## Classification Player's team with color of jersey by K-Means 

### Preparing the dataset

In [3]:
import sportslabkit as slk
from sportslabkit.logger import set_log_level
import tqdm as notebook_tqdm

dataset_path = slk.datasets.get_path("wide_view")
path_to_csv = sorted(dataset_path.glob("annotations/*.csv"))[0]
path_to_mp4 = sorted(dataset_path.glob("videos/*.mp4"))[0]

root = slk.utils.get_git_root()
cam = slk.Camera(path_to_mp4)

# For the sake of speed, we'll only use the first 10 frames
n_frames = 2
frames = cam[:n_frames]

bbdf_gt = slk.load_df(path_to_csv)
# TODO: Hopefully we can get rid of this 
if bbdf_gt.index[0] == 0:
    bbdf_gt.index += 1
bbdf_gt = bbdf_gt[:n_frames]


In [4]:
# Team's Information used to predict player's team
from typing import Tuple
from dataclasses import dataclass

# Set team_id and color of jersey
@dataclass
class Team:
    team_name: str
    team_id: int
    jersey_color: Tuple[int, int, int]
    gk_jersey_color: Tuple[int, int, int]

team1 = Team(
    team_name = "Name of Team1",
    team_id = 0,
    jersey_color = (255, 0, 0),
    gk_jersey_color = (0,0,0)
)

team2 = Team(
    team_name = "Name of Team2",
    team_id = 1,
    jersey_color = (0, 255, 0),
    gk_jersey_color = (0, 0, 255)
)

In [5]:
import cv2
import numpy as np
import matplotlib as plt
from sklearn.cluster import KMeans

class DominantColors:
    """
    Find second most important color of a subimage.
    First color is arguably green (color of the soccer pitch)
    """

    CLUSTERS = None
    IMAGE = None
    COLORS = None
    LABELS = None

    def __init__(self, img, clusters=3):
        self.CLUSTERS = clusters
        self.img = img.copy()

    def dominant_colors(self):
        """
        Perform K-means on the RGB space to find 2 main colors
        """
        if self.img is None:
            print("Error: image is None")
            return
        if self.img.size == 0:
            print("Error: image is empty")
            return
        # convert to rgb from bgr
        img = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)

        # reshaping to a list of pixels
        img = img.reshape((img.shape[0] * img.shape[1], 3))

        # save image after operations
        self.IMAGE = img

        # using k-means to cluster pixels
        kmeans = KMeans(n_clusters=self.CLUSTERS, n_init=1, random_state=1, max_iter=10)
        kmeans.fit(img)

        # the cluster centers are our dominant colors.
        self.COLORS = kmeans.cluster_centers_


        # save labels
        self.LABELS = kmeans.labels_

        labels, counts = np.unique(kmeans.labels_, return_counts=True)

        self.sub_COLORS = self.COLORS[np.argsort(counts)].astype(int)

        hsv_similarity_home = hsv_color_similarity((self.sub_COLORS[1][0], self.sub_COLORS[1][1], self.sub_COLORS[1][2]), team1.jersey_color)
        hsv_similarity_away = hsv_color_similarity((self.sub_COLORS[1][0], self.sub_COLORS[1][1], self.sub_COLORS[1][2]), team2.jersey_color)
        if hsv_similarity_home > hsv_similarity_away:
            return Color(team1.jersey_color[2],team1.jersey_color[1],team1.jersey_color[0])
        else:
            return Color(team2.jersey_color[2],team2.jersey_color[1],team2.jersey_color[0])
        # returning after converting to integer from float


    def plot_histogram(self):
        """
        Plot color histogram for debuggging only
        """

        # labels form 0 to no. of clusters
        numLabels = np.arange(0, self.CLUSTERS + 1)

        # create frequency count tables
        (hist, _) = np.histogram(self.LABELS, bins=numLabels)
        hist = hist.astype("float")
        hist /= hist.sum()

        # appending frequencies to cluster centers
        colors = self.COLORS

        # descending order sorting as per frequency count
        colors = colors[(-hist).argsort()]
        hist = hist[(-hist).argsort()]

        # creating empty chart
        chart = np.zeros((50, 500, 3), np.uint8)
        start = 0

        # creating color rectangles
        for i in range(self.CLUSTERS):
            end = start + hist[i] * 500

            # getting rgb values
            r = colors[i][0]
            g = colors[i][1]
            b = colors[i][2]

            # using cv2.rectangle to plot colors
            cv2.rectangle(chart, (int(start), 0), (int(end), 50), (r, g, b), -1)
            start = end

        fig, (ax1, ax2) = plt.subplots(1, 2)
        ax1.axis("off")
        image = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)
        ax1.imshow(np.array(image))

        # display chart
        # ax2.figure()
        ax2.axis("off")
        ax2.imshow(chart)
        plt.show()


@dataclass(frozen=True)
class Color:
    r: int
    g: int
    b: int

    @property
    def bgr_tuple(self) -> Tuple[int, int, int]:
        return self.r, self.g, self.b


import colorsys
import math

def euclidean_distance(color1, color2):
    """
    RGB色空間での2つの色のEuclidean距離を計算する関数
    """
    return math.sqrt((color1[0] - color2[0]) ** 2 + (color1[1] - color2[1]) ** 2 + (color1[2] - color2[2]) ** 2)


def hsv_color_similarity(color1, color2):
    """
    2つの色のHSV表現を使用して類似度を計算する関数
    """
    # Convert from RGB to hsv
    hsv1 = colorsys.rgb_to_hsv(color1[0] / 255.0, color1[1] / 255.0, color1[2] / 255.0)
    hsv2 = colorsys.rgb_to_hsv(color2[0] / 255.0, color2[1] / 255.0, color2[2] / 255.0)

    # 色相、彩度、明度の差を計算
    hue_difference = min(abs(hsv1[0] - hsv2[0]), 1.0 - abs(hsv1[0] - hsv2[0]))
    saturation_difference = abs(hsv1[1] - hsv2[1])
    value_difference = abs(hsv1[2] - hsv2[2])

    # Culculate similarity(range:0~1)
    similarity = 1.0 - ((hue_difference + saturation_difference + value_difference) / 3.0)

    return similarity

In [6]:
from sportslabkit.mot import TeamTracker
import numpy as np 

slk.logger.set_log_level('INFO')
det_model = slk.detection_model.load(
    model_name='yolov8',
    model=root/'models/yolov8/Soccer_SideView.pt',
    conf=0.25,
    iou=0.6,
    imgsz=640,
    device='mps',
    classes=0,
    augment=True,
    max_det=35
)

image_model = slk.image_model.load(
    model_name='mobilenetv2_x1_0',
    image_size=(32,32),
    device='mps'
)

motion_model = slk.motion_model.load(
    model_name='SingleTargetLinear',
)
# motion_model = slk.motion_model.load(
#     model_name='SingleTargetLSTM',
#     model='/Users/atom/Github/SoccerTrack/models/teamtrack/LSTM-F_Soccer_Tsukuba3-epoch=79-val_nll_loss=-2.74.ckpt',
# )

keypoint_json = root / 'notebooks/02_user_guide/assets/soccer_keypoints.json'
cam.source_keypoints, cam.target_keypoints = slk.utils.load_keypoints(keypoint_json)

# calibration model return a 3x3 homography matrix for each frame
calibration_model = slk.calibration_model.load(
    model_name='DummyCalibrationModel',
    homographies=cam.H,
    mode='constant'
)

first_matching_fn = slk.matching.MotionVisualMatchingFunction(
    motion_metric=slk.metrics.EuclideanCMM2D(use_pred_pt=True),
    motion_metric_gate=0.2,
    visual_metric=slk.metrics.CosineCMM(),
    visual_metric_gate=0.2,
    beta=0.9,
)

second_matching_fn = slk.matching.SimpleMatchingFunction(
    metric=slk.metrics.EuclideanCMM2D(use_pred_pt=True),
    gate=0.9,
)

# team_detection_callback = slk.callbacks.TeamDetectionCallback(classication_model=TeamClassifier())

class PrintingCallback(slk.callbacks.Callback):
    def on_track_sequence_start(self, tracker):
        tracklets = tracker.alive_tracklets + tracker.dead_tracklets
        print(f"Tracking started with {len(tracklets)} tracklets")
    
    def on_track_sequence_end(self, tracker):
        tracklets = tracker.alive_tracklets + tracker.dead_tracklets
        print(f"Tracking ended with {len(tracklets)} tracklets")

callbacks = [PrintingCallback()]

tracker = TeamTracker(
    detection_model=det_model,
    image_model=image_model,
    motion_model=motion_model,
    calibration_model=calibration_model,
    first_matching_fn=first_matching_fn,
    second_matching_fn=second_matching_fn,
    detection_score_threshold=0.6,
    max_staleness=2,
    min_length=2,
    callbacks=callbacks,
)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/minorisugimura/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sportslabkit/utils/models/yolov8/Soccer_SideView.pt'

In [ ]:
tracker.track(frames)[0]

In [ ]:
def plot_image(image: np.ndarray, size: int = 12) -> None:
    plt.figure(figsize=(size, size))
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)
    plt.tick_params(bottom=False, left=False, right=False, top=False)
    plt.imshow(image[...,::-1])
    plt.show()

for i in range(len(frames)):
    frame = frames[0]

    plot_image(frame, 8)